In [6]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [7]:
data = pd.read_csv("monte_carlo.csv")

In [12]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        p = np.zeros((2,2,n))

        for a_j in [0,1]:
            util1 = np.dot(self.exog[:,0:k], params[0:k])  + params[k]*a_j
            util2 = np.dot(self.exog[:,k:], params[k+1:2*k+1]) + params[2*k+1]*a_j
            p[0,a_j,:] = 1 /(1 + np.exp(util1))
            p[1,a_j,:] = 1 /(1 + np.exp(util2))
        
        lamb = .5
        
        #solve for probablity of nash
        mult_eq = np.maximum( ( p[0,1,:]  - p[0,0,:] )*(  p[1,1,:] - p[1,0,:] ),0)
        prob01 = ( p[0,1,:] )*( 1 - p[1,0,:] ) - (1-lamb)*mult_eq
        prob10 = ( 1 - p[0,0,:] )*( p[1,1,:] ) - (lamb)*mult_eq
        prob00 = p[0,0,:] * p[1,0,:]
        prob11 = ( 1 - p[0,1,:] )*( 1 - p[1,1,:] )

        #compute empirical likelihood
        p00 = (1 - self.endog[:,0])*(1 - self.endog[:,1])
        p11 = self.endog[:,0]*self.endog[:,1]
        p10 = self.endog[:,0]* (1 - self.endog[:,1])
        p01 = (1 - self.endog[:,0]) * self.endog[:,1]

        ll = p00 * prob00 + p11 *prob11 + p01 * prob01 + p10 * prob10
        
       
        return -np.log(np.maximum(ll,1e-12)).sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+2)
        return super(NashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = data.shape[0]
y = data[['y1','y2']]
x = data[['x1','x2']]

model = NashLogit(y,x)
model = model.fit()
print(model.summary(xname=["b11","d1","b12","d2"]))

Optimization terminated successfully.
         Current function value: 0.966744
         Iterations: 131
         Function evaluations: 231
                              NashLogit Results                               
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -966.74
Model:                      NashLogit   AIC:                             1937.
Method:            Maximum Likelihood   BIC:                             1947.
Date:                Wed, 07 Apr 2021                                         
Time:                        15:55:55                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------